In [64]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')

from IPython.core.display import Image
import uuid 

import gc

In [65]:
#R call

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')
base = importr('base')
stats = importr('stats')

In [66]:
#Call custom Python module
import parametric as param

In [67]:
def is_Hermitian(m):
    #Set missing to zero prior to checking symmetry
    m[np.isnan(m)] = 0.

    try:
        return np.allclose(np.transpose(m,(1,0)), m)
    except:
        return False

In [88]:
def fastclime_py(x,lambda_min=0.1,nlambda=50):

    cov_input = 1
    SigmaInput = x.copy()
    
    if not is_Hermitian(SigmaInput):
        SigmaInput = np.corrcoef(SigmaInput.T)
        cov_input = 0
    
    
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param.mainfunc(SigmaInput,lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    lambdamtx = mu#mu[:maxnlambda,]
    mu = None

    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
    for i in range(maxnlambda):
        icovlist[:,:,i] = iicov[i,:].reshape((d,d))
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx.T, icovlist
    

In [92]:
#Generate data

L = fastclime.fastclime_generator(n = 5, d = 7)
pydat = np.array(L.rx2('data'))

Generating data from the multivariate normal distribution with the random graph structure....done.


In [93]:
Pyout = fastclime_py(pydat,0.1)
print Pyout[4]
#%timeit -n1 -r1 fastclime_py(pydat,0.1)

[[ 1.      0.2919  0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.452   0.      0.      0.      0.      0.      0.
   0.3747  0.      0.      0.      0.      0.      0.      0.4796  0.      0.
   0.      0.      0.      0.      0.3903  0.2261  0.      0.      0.      0.
   0.    ]
 [ 0.3965  0.2278  0.      0.      0.      0.      0.      1.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.4372  0.      0.      0.      0.      0.      0.
   0.3746  0.      0.      0.      0.      0.      0.      0.445   0.      0.
   0.      0.      0.      0.      0.2893  0.      0.      0.      0.      0.
   0.    ]
 [ 0.3919  0.      0.      0.      0.      0.      0.      0.4494  0.      0.
   0.      0.      0.      0.      1.      0.      0.      0.      0.      0.
   0.      0.      0.415   0.      0.      0.      0.     

In [82]:
(50-Pyout[3]+1)/5-1

7.2000

In [62]:
Pyout[4][1][::10]

array([ 0.4659,  0.4339,  0.2814,  0.4659,  0.4339])

In [63]:
Pyout[4][0][1::10]

array([ 0.345 ,  0.3608,  0.1883,  0.2936,  0.3323])

In [ ]:
#Compare results to R

In [94]:
#%timeit -n1 -r1 fastclime.fastclime(L.rx2('data'),0.1)
Rout = fastclime.fastclime(L.rx2('data'),0.1)
#np.array(Rout.rx2('icovlist')[16])
np.array(Rout.rx2('lambdamtx'))

Allocating memory 
start recovering 
preparing precision and path matrix list 
Done! 


array([[ 1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ],
       [ 0.3965,  0.4494,  0.4494,  0.4828,  0.468 ,  0.4828,  0.3903],
       [ 0.3919,  0.3768,  0.3915,  0.4748,  0.3748,  0.4796,  0.2893],
       [ 0.3904,  0.351 ,  0.3817,  0.4544,  0.3747,  0.445 ,  0.2674],
       [ 0.3896,  0.3477,  0.3814,  0.452 ,  0.3746,  0.4146,  0.2484],
       [ 0.3761,  0.3445,  0.    ,  0.4372,  0.3738,  0.4012,  0.2435],
       [ 0.3131,  0.    ,  0.    ,  0.415 ,  0.3727,  0.    ,  0.2419],
       [ 0.2919,  0.    ,  0.    ,  0.4011,  0.    ,  0.    ,  0.2309],
       [ 0.2278,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.2261]])

In [ ]:
#It seems that there is some matrix manipulation that R/C is doing behind the scenes that's being modified in Python. 


In [ ]:
###Draw network graph

# """
# Draw a graph with matplotlib.
# You must have matplotlib for this to work.
# """
# try:
#     import matplotlib.pyplot as plt
# except:
#     raise

# import networkx as nx

# G=nx.path_graph(8)
# nx.draw(G)
# plt.savefig("simple_path.png") # save as png
# plt.show() # display